In [ ]:
# Coinbase API Sanity Check

This notebook provides basic API testing and verification for the Coinbase rebalancing agent.


In [ ]:
import os
import sys
from dotenv import load_dotenv
from coinbase.rest import RESTClient

# Load environment variables
load_dotenv()

# Add the agent module to the path
sys.path.append('../')

# Initialize Coinbase client
client = RESTClient(
    api_key=os.getenv("CB_API_KEY"),
    api_secret=os.getenv("CB_API_SECRET"),
    base_url=os.getenv("CB_API_URL", "https://api.coinbase.com")
)

print("✅ Coinbase client initialized")


In [ ]:
from agent.selector import build_target_weights, fetch_usd_products

# Test dynamic selector with top 10 assets
print("🔍 Testing dynamic selector...")
tgt = build_target_weights(client, top_n=10)
print(f"📊 Generated target weights for {len(tgt)} assets:")
for asset, weight in tgt.items():
    print(f"  {asset}: {weight:.4f} ({weight*100:.2f}%)")

print(f"\n💰 Total allocation: {sum(tgt.values()):.4f} ({sum(tgt.values())*100:.2f}%)")
print(f"💵 Cash buffer: {1 - sum(tgt.values()):.4f} ({(1 - sum(tgt.values()))*100:.2f}%)")


In [ ]:
# Basic API connectivity test
print("🌐 Testing API connectivity...")

try:
    # Test basic API call
    products = client.list_products(limit=5)
    print(f"✅ API connection successful!")
    print(f"📦 Found {len(products['products'])} products (showing first 5)")
    
    for product in products['products']:
        print(f"  - {product['product_id']}: {product['status']}")
        
except Exception as e:
    print(f"❌ API connection failed: {e}")


In [ ]:
# Coinbase API Sanity Check

This notebook verifies API connectivity and demonstrates portfolio visualization capabilities.


In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add parent directory to path to import agent modules
sys.path.append('..')

from agent import config, utils
from agent.utils import get_client, fetch_nav_and_positions, calculate_current_weights

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")


In [ ]:
## 1. API Configuration Check


In [ ]:
# Check API configuration
print("🔧 API Configuration:")
print(f"  API URL: {config.API_URL}")
print(f"  API Key: {'✅ Set' if config.API_KEY else '❌ Not set'}")
print(f"  API Secret: {'✅ Set' if config.API_SECRET else '❌ Not set'}")
print(f"  API Passphrase: {'✅ Set' if config.API_PASSPHRASE else '❌ Not set'}")
print(f"  Portfolio ID: {'✅ Set' if config.PORTFOLIO_ID else '❌ Not set'}")
print(f"  Dry Run: {config.DRY_RUN}")

# Validate configuration
try:
    config.validate_config()
    print("\n✅ Configuration is valid!")
except Exception as e:
    print(f"\n❌ Configuration error: {e}")
    print("💡 Make sure to copy .env.example to .env and fill in your API credentials")


In [ ]:
## 2. Portfolio Visualization


In [ ]:
# Create sample portfolio data for visualization
sample_portfolio = {
    'BTC-USD': 0.15,   # 15%
    'ETH-USD': 0.20,   # 20%
    'SOL-USD': 0.12,   # 12%
    'ADA-USD': 0.10,   # 10%
    'DOT-USD': 0.08,   # 8%
    'AVAX-USD': 0.08,  # 8%
    'LINK-USD': 0.07,  # 7%
    'ATOM-USD': 0.07,  # 7%
    'ALGO-USD': 0.06,  # 6%
    'MATIC-USD': 0.07, # 7%
}

# Create comparison with target weights
comparison_data = []
for symbol, current_weight in sample_portfolio.items():
    target_weight = float(config.TARGET_WEIGHTS.get(symbol, 0))
    comparison_data.append({
        'Asset': symbol,
        'Current': current_weight * 100,
        'Target': target_weight * 100,
        'Difference': (target_weight - current_weight) * 100
    })

df_comparison = pd.DataFrame(comparison_data)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Portfolio composition pie chart
ax1.pie(df_comparison['Current'], labels=df_comparison['Asset'], autopct='%1.1f%%')
ax1.set_title('Current Portfolio Composition', fontsize=14, fontweight='bold')

# Current vs Target weights
df_comparison.set_index('Asset')[['Current', 'Target']].plot(kind='bar', ax=ax2, color=['lightblue', 'orange'])
ax2.set_title('Current vs Target Weights', fontsize=14, fontweight='bold')
ax2.set_ylabel('Weight (%)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Portfolio visualization complete!")
print(f"📈 Total assets: {len(df_comparison)}")
print(f"🎯 Target weights configured: {len(config.TARGET_WEIGHTS)}")
